In [48]:
import pandas as pd
from collections import defaultdict
import numpy as np

In [40]:
from pathlib import Path

file_dir = Path.cwd().parent / "euro-interconnector-pipeline" / "data" / "raw" / "fms" / "physical_flows"
files = [f for f in file_dir.iterdir()]
print(files[0])
df_sample = pd.read_csv(files[0], sep="\t")
print(df_sample.columns)
df_sample

c:\Users\zkong\Desktop\energy-fundies\euro-interconnector-pipeline\data\raw\fms\physical_flows\2014_04_PhysicalFlows_12.1.G_r3.csv
Index(['DateTime(UTC)', 'ResolutionCode', 'OutAreaCode', 'OutAreaDisplayName',
       'OutAreaTypeCode', 'OutAreaMapCode', 'InAreaCode', 'InAreaDisplayName',
       'InAreaTypeCode', 'InAreaMapCode', 'Flow[MW]', 'UpdateTime(UTC)'],
      dtype='object')


,DateTime(UTC),ResolutionCode,OutAreaCode,OutAreaDisplayName,OutAreaTypeCode,OutAreaMapCode,InAreaCode,InAreaDisplayName,InAreaTypeCode,InAreaMapCode,Flow[MW],UpdateTime(UTC)
0,2014-04-07 23:00:00,PT60M,10YGB----------A,GB,BZN/CTA,GB,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,0.00,2025-08-28 12:08:11
1,2014-04-08 00:00:00,PT60M,10YGB----------A,GB,BZN/CTA,GB,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,0.00,2025-08-28 12:08:12
2,2014-04-08 01:00:00,PT60M,10YGB----------A,GB,BZN/CTA,GB,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,0.00,2025-08-28 12:08:12
3,2014-04-08 02:00:00,PT60M,10YGB----------A,GB,BZN/CTA,GB,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,0.00,2025-08-28 12:08:12
4,2014-04-08 03:00:00,PT60M,10YGB----------A,GB,BZN/CTA,GB,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,0.00,2025-08-28 12:08:12
...,...,...,...,...,...,...,...,...,...,...,...,...
91,2014-04-08 18:00:00,PT60M,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,10Y1001A1001A92E,United Kingdom (UK),CTY,GB,152.55,2025-08-28 12:08:12
92,2014-04-08 19:00:00,PT60M,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,10Y1001A1001A92E,United Kingdom (UK),CTY,GB,152.55,2025-08-28 12:08:12
93,2014-04-08 20:00:00,PT60M,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,10Y1001A1001A92E,United Kingdom (UK),CTY,GB,152.55,2025-08-28 12:08:12
94,2014-04-08 21:00:00,PT60M,10YIE-1001A00010,Ireland (IE),CTA/CTY,IE,10Y1001A1001A92E,United Kingdom (UK),CTY,GB,152.55,2025-08-28 12:08:12


## Investigate Resolution Code

In [ ]:
unique_resolution_code = set()
import polars as pl

for file in files:
    s = pl.read_csv(file, separator="\t", columns=["ResolutionCode"]).get_column(
        "ResolutionCode"
    )
    unique_resolution_code.update(s.drop_nulls().unique().to_list())
print(unique_resolution_code)

## Investigate OutArea codes

In [ ]:
OutAreaCols = ['OutAreaCode', 'OutAreaDisplayName', 'OutAreaTypeCode', 'OutAreaMapCode']
OutArea_map = defaultdict(lambda: set())
for file in files:
    df_current = pl.read_csv(file, separator="\t", columns=OutAreaCols)
    outareacode_rows = set(list(df_current[OutAreaCols].rows()))
    for row in outareacode_rows:
        OutArea_map[row[0]].update([tuple(row[1:])])

In [47]:
for k, v in OutArea_map.items():
    if len(v) > 1:
        print(k, v)
print("If nothing got printed, means we can use OutAreaCode as the unique identifier for the area")


If nothing got printed, means we can use OutAreaCode as the unique identifier for the area


## Investigate InArea Codes

In [ ]:
InAreaCols = ["InAreaCode", "InAreaDisplayName", "InAreaTypeCode", "InAreaMapCode"]

InArea_map = defaultdict(lambda: set())
for file in files:
    df_current = pl.read_csv(file, separator="\t", columns=InAreaCols)
    in_area_code_rows = set(list(df_current[InAreaCols].rows()))
    for row in in_area_code_rows:
        InArea_map[row[0]].update([tuple(row[1:])])

for k, v in InArea_map.items():
    if len(v) > 1:
        print(k, v)
print(
    "If nothing got printed, means we can use InAreaCode as the unique identifier for the area"
)

If nothing got printed, means we can use InAreaCode as the unique identifier for the area
